# Loading Data

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install kaggle --upgrade
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -1ha kaggle.json

In [ ]:
!kaggle competitions download -c mercari-price-suggestion-challenge

In [ ]:
# 확인
!ls
# 압축 풀기
!unzip mercari-price-suggestion-challenge
!unzip test_stg2.tsv

In [ ]:
!pip install wordcloud
!pip install koreanize-matplotlib
#!pip install lightgbm
!pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from scipy.sparse import hstack
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.stats import uniform, randint
from sklearn.feature_selection import SelectKBest, f_regression
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import koreanize_matplotlib
import warnings
warnings.filterwarnings('ignore')
from sklearn.svm import SVR
import xgboost as xgb

In [ ]:
# Step 1: Install 7-zip
!apt-get install p7zip-full

# Step 2: Unzip the .7z file
!7z x /content/train.tsv.7z -o/content/
!7z x /content/test.tsv.7z -o/content/
!7z x /content/test_stg2.tsv.7z -o/content/


# Step 3: Read the unzipped files with pandas
#import pandas as pd

train = pd.read_csv("/content/train.tsv", sep='\t', encoding='utf-8')
test = pd.read_csv("/content/test.tsv", sep='\t', encoding='utf-8')
test_stg2 = pd.read_csv("/content/test_stg2.tsv", sep='\t', encoding='utf-8')

# Display the first few rows of the train and test datasets
print(train.head())
print(test.head())
print(test_stg2.head())

# 데이터 살펴보기

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
test_stg2.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
test_stg2.info()

In [ ]:
print(train.shape)
print(test.shape)
print(test_stg2.shape)

In [ ]:
## 가독성 있게
pd.set_option('display.float_format', lambda x: '%.3f' % x)
train.describe()

In [ ]:
# test.describe()

In [ ]:
# test_stg2.describe()

In [ ]:
train.isnull().sum()  #브랜드 카테고리 설명 결측치가 있다.
## category_name, brand_name에서 대부분의 결측치가 보였다.
## item_description은 6개니까 제거하는 방향으로 가야겠다.

## 저 데이터들을 마냥 제거하기엔 너무 많다. 다른 방법이 있을 것이다.
## 우선 EDA에 걸림돌이 되지 않는 선에서 분석을 진행한다.

In [ ]:
test.isnull().sum()

In [ ]:
test_stg2.isnull().sum()

# 데이터 전처리

In [ ]:
## 레이블 인코딩 함수
def label_encode(df, columns):
    le = LabelEncoder()
    for col in columns:
        df[col] = le.fit_transform(df[col].astype(str))
    return df, le

In [ ]:
## RMSLE 계산
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [ ]:
# 데이터 전처리 함수들

# 입력 텍스트를 전처리하는 함수
def preprocess(Input_text):
    # 자주 사용되는 정규 표현식 패턴을 미리 컴파일
    HTML_CLEANER = re.compile('<.*?>')
    EMAIL_CLEANER = re.compile(r'\S*@\S*\s?')
    URL_CLEANER = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    PARENTHESES_CLEANER = re.compile(r"[\(\[].*?[\)\]]")
    WORD_COLON_CLEANER = re.compile(r'\w+:\s?')
    NUMBER_CLEANER = re.compile(r'[0-9]+')
    NON_ALPHA_CLEANER = re.compile(r'[^a-zA-Z]+')

    # 입력 텍스트의 처음 60개 단어만 사용
    cleantext = ' '.join([w for w in Input_text.split()[:60]])

    # HTML 태그, 이메일, URL 제거
    cleantext = HTML_CLEANER.sub('', cleantext)
    cleantext = EMAIL_CLEANER.sub(' ', cleantext)
    cleantext = URL_CLEANER.sub(' ', cleantext)

    # 괄호 안의 내용 제거
    cleantext = PARENTHESES_CLEANER.sub("", cleantext)

    # 단어 뒤의 콜론 제거
    cleantext = WORD_COLON_CLEANER.sub('', cleantext)

    # 축약형 처리
    contractions = {
        r"\'m": " am", r"n\'t": " not", r"\'t": " not", r"\'s": " is",
        r"\'ve": " have", r"\'re": " are", r"\'ll": " will",
        r"won\'t": "will not", r"can\'t": "can not", r"\'d": " would",
        r"y\'all": "you all", r"ain\'t": "is not", r"aren\'t": "are not",
        r"couldn\'t": "could not", r"didn\'t": "did not", r"doesn\'t": "does not",
        r"hadn\'t": "had not", r"hasn\'t": "has not", r"haven\'t": "have not",
        r"he\'d": "he would", r"he\'ll": "he will", r"he\'s": "he is",
        r"i\'d": "i would", r"i\'ll": "i will", r"i\'m": "i am",
        r"isn\'t": "is not", r"it\'s": "it is", r"let\'s": "let us",
        r"mightn\'t": "might not", r"mustn\'t": "must not", r"shan\'t": "shall not",
        r"she\'d": "she would", r"she\'ll": "she will", r"she\'s": "she is",
        r"shouldn\'t": "should not", r"that\'s": "that is", r"there\'s": "there is",
        r"they\'d": "they would", r"they\'ll": "they will", r"they\'re": "they are",
        r"they\'ve": "they have", r"we\'d": "we would", r"we\'re": "we are",
        r"weren\'t": "were not", r"what\'ll": "what will", r"what\'re": "what are",
        r"what\'s": "what is", r"what\'ve": "what have", r"where\'s": "where is",
        r"who\'d": "who would", r"who\'ll": "who will", r"who\'re": "who are",
        r"who\'s": "who is", r"who\'ve": "who have", r"won\'t": "will not",
        r"wouldn\'t": "would not", r"you\'d": "you would", r"you\'ll": "you will",
        r"you\'re": "you are", r"you\'ve": "you have"
    }

    for contraction, expansion in contractions.items():
        cleantext = re.sub(contraction, expansion, cleantext)

    # 숫자 제거
    cleantext = NUMBER_CLEANER.sub('', cleantext)

    # 소문자로 변환
    cleantext = cleantext.lower()

    # 알파벳이 아닌 문자를 공백으로 대체
    cleantext = NON_ALPHA_CLEANER.sub(' ', cleantext)

    # 연속된 공백을 하나의 공백으로 치환
    cleantext = re.sub(r'\s+', ' ', cleantext).strip()

    return cleantext

In [ ]:
## 전처리
for df in [train, test, test_stg2]:
    df['category_name'] = df['category_name'].fillna('Unknown/Unknown/Unknown')
    df[['category1', 'category2', 'category3']] = df['category_name'].str.split('/', n=2, expand=True)
    df[['category1', 'category2', 'category3']] = df[['category1', 'category2', 'category3']].fillna('Unknown')
    df = df.drop(columns=['category_name'])
    df['brand_name'] = df['brand_name'].fillna('No Brand')
    df['item_description'] = df['item_description'].fillna('No description')

    for col in ['name', 'item_description', 'brand_name', 'category1', 'category2', 'category3']:
        df[col] = df[col].apply(preprocess)


In [ ]:
def robust_label_encode(train, test, test_stg2, categorical_cols):
    le_dict = {}
    for col in categorical_cols:
        le = LabelEncoder()

        ## 모든 데이터셋의 유니크한 값을 결합.
        unique_values = pd.concat([train[col], test[col], test_stg2[col]]).unique().tolist()
        unique_values.append('Unknown')

        ## 'Unknown' 추가
        unique_values.append('Unknown')

        ## 레이블 인코더 학습
        le.fit(unique_values)

        ## 각 데이터셋에 레이블 인코딩 적용
        train[col] = le.transform(train[col].fillna('Unknown'))
        test[col] = le.transform(test[col].fillna('Unknown'))
        test_stg2[col] = le.transform(test_stg2[col].fillna('Unknown'))

        le_dict[col] = le

    return train, test, test_stg2, le_dict

In [ ]:
print(df.shape)
df.isnull().sum()

In [ ]:
## 가격이 0 이상인 데이터만 추출 (훈련 데이터만)
train = train[train['price'] > 0]

In [ ]:
## 레이블 인코딩
categorical_cols = ['brand_name', 'category1', 'category2', 'category3']
train, test, test_stg2, le_dict = robust_label_encode(train, test, test_stg2, categorical_cols)

print("Label encoding completed successfully.")
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

def preprocess_text(df, text_columns):
    for col in text_columns:
        df[col] = df[col].fillna('').astype(str)
    return df

In [ ]:
## 텍스트 전처리 함수 적용
text_columns = ['name', 'item_description']
train = preprocess_text(train, text_columns)
test = preprocess_text(test, text_columns)
test_stg2 = preprocess_text(test_stg2, text_columns)

In [ ]:
## TF-IDF 벡터화
vectorizer_name = TfidfVectorizer(max_features=1000)
vectorizer_desc = TfidfVectorizer(min_df=10, max_features=50000)

In [ ]:
## 훈련 데이터 벡터화
train_name_tfidf = vectorizer_name.fit_transform(train['name'])
train_id_tfidf = vectorizer_desc.fit_transform(train['item_description'])

In [ ]:
## 테스트 데이터 벡터화
test_name_tfidf = vectorizer_name.transform(test['name'])
test_id_tfidf = vectorizer_desc.transform(test['item_description'])


In [ ]:
## 2단계 테스트 데이터 벡터화
test_stg2_name_tfidf = vectorizer_name.transform(test_stg2['name'])
test_stg2_id_tfidf = vectorizer_desc.transform(test_stg2['item_description'])

print("TF-IDF vectorization completed successfully.")

In [ ]:
# 특성 결합
X = hstack([train_name_tfidf, train_id_tfidf, train[categorical_cols]])
X_test = hstack([test_name_tfidf, test_id_tfidf, test[categorical_cols]])
X_test_stg2 = hstack([test_stg2_name_tfidf, test_stg2_id_tfidf, test_stg2[categorical_cols]])

y = train['price'].values
# y = np.log1p(train['price']).values

In [ ]:
# 특성 선택
selector = SelectKBest(f_regression, k=1000)  # 상위 1000개 특성 선택
X_selected = selector.fit_transform(X, y)
X_test_selected = selector.transform(X_test)
X_test_stg2_selected = selector.transform(X_test_stg2)

In [ ]:
# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [ ]:
# 모델링 및 평가 함수들

# 모델 성능을 평가하는 함수
def evaluate_model(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    medae = median_absolute_error(y_test, y_pred)
    me = max_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    ev = explained_variance_score(y_test, y_pred)

    result = {
        'Mean Absolute Error': mae,
        'Mean Squared Error': mse,
        'Mean Absolute Percentage Error': mape,
        'Median Absolute Error': medae,
        'Max Error': me,
        'R^2': r2,
        'Explained Variance': ev,
    }
    return result

In [ ]:
# 하이퍼파라미터 탐색 공간 정의
param_distributions = {
    'Ridge': {'alpha': uniform(0.1, 10)},
    'RandomForest': {
        'n_estimators': randint(100, 500),
        'max_depth': randint(5, 15),
        'min_samples_split': randint(2, 11)
    },
    'XGBoost': {
        'n_estimators': randint(100, 300), # 트리 수
        'max_depth': randint(3, 7), # 트리 최대 깊이
        'learning_rate': uniform(0.01, 0.2), # 학습률
        'subsample': uniform(0.7, 0.3), # 각 트리에 사용할 샘플 비율
        'colsample_bytree': uniform(0.7, 0.3), # 각 트리에 사용할 컬럼 비율
        'gamma': uniform(0, 2), # 리프 노드 분할에 필요한 최소 손실감소량
        'min_child_weight': randint(1, 5) # 리프 노드에 필요한 최소 가중치
    }
}

In [ ]:
# 모델 정의
models = {
    'Ridge': Ridge(),
    'RandomForest': RandomForestRegressor(random_state = 42),
    'XGBoost': xgb.XGBRegressor(random_state = 42)
}



# 하이퍼파라미터 튜닝 및 모델 학습
tuned_models = {}
predictions = {}
predictions_stg2 = {}

for name, model in models.items():
    print(f"Tuning {name}...")
    if name == 'LightGBM':
        # LightGBM의 경우 GPU 사용을 위한 추가 설정
        fit_params = {'verbose': 0, 'callbacks': [lgb.early_stopping(50)]}
        random_search = RandomizedSearchCV(
            model, param_distributions[name], n_iter=20, cv=3,
            scoring='neg_mean_squared_log_error', n_jobs=1, random_state=42,
            fit_params=fit_params
        )
    else:
        random_search = RandomizedSearchCV(
            model, param_distributions[name], n_iter=20, cv=3,
            scoring='neg_mean_squared_log_error', n_jobs=-1, random_state=42
        )

    random_search.fit(X_train, np.log1p(y_train))

    best_model = random_search.best_estimator_
    tuned_models[name] = best_model

    val_pred = np.expm1(best_model.predict(X_val))
    test_pred = np.expm1(best_model.predict(X_test_selected))
    test_stg2_pred = np.expm1(best_model.predict(X_test_stg2_selected))

    print(f"{name} RMSLE: {rmsle(y_val, val_pred)}")
    print(f"Best parameters: {random_search.best_params_}")

    predictions[name] = test_pred
    predictions_stg2[name] = test_stg2_pred

In [ ]:
# 앙상블 (단순 평균)
ensemble_pred = np.mean(list(predictions.values()), axis=0)
ensemble_pred_stg2 = np.mean(list(predictions_stg2.values()), axis=0)

# Submission 파일 생성
submission = pd.read_csv('sample_submission.csv')
submission['price'] = ensemble_pred
submission.to_csv('ensemble_submission.csv', index=False)

submission_stg2 = pd.read_csv('sample_submission_stg2.csv')
submission_stg2['price'] = ensemble_pred_stg2
submission_stg2.to_csv('ensemble_submission_stg2.csv', index=False)

print("Submission files created: ensemble_submission.csv and ensemble_submission_stg2.csv")


# EDA

In [ ]:
plt.figure(figsize=(15,5))

# 첫 번째 서브플롯: 가격 분포
plt.subplot(1,2,1)
plt.hist(df['price'], bins=100 , edgecolor='white')  # range=[0,200]
plt.title('가격 분포')
plt.xlabel('가격')
plt.ylabel("제품 수")

# 두 번째 서브플롯: 로그 변환된 가격 분포
plt.subplot(1,2,2)
#log_price = np.log(df['price']+1)
log_price = np.log1p(df['price'])
plt.hist(log_price1, bins=100, edgecolor='white')
plt.title("로그 변환된 가격 분포")
plt.xlabel("log(가격+1)")
plt.ylabel("제품 수")

plt.tight_layout()  # 서브플롯 간 간격 조정
plt.show()

### Category

In [ ]:
# category1의 빈도수 계산
count = df['category1'].value_counts()

# 데이터프레임으로 변환
count_df = count.reset_index()
count_df.columns = ['category1', 'count']

plt.figure(figsize=(16, 8))
sns.barplot(x='category1', y='count', data=count_df, palette='husl')
plt.title("카테고리1의 항목 순위", fontsize=20)
plt.xlabel('카테고리1', fontsize=14)
plt.ylabel('빈도수', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

- 상위 3개의 카테고리 : 여성, 뷰티, 아동

In [ ]:
# category2의 빈도수 계산
count = df['category2'].value_counts()

# 데이터프레임으로 변환
count_df = count.reset_index()
count_df.columns = ['category2', 'count']
top_20_count_df = count_df.head(20)


plt.figure(figsize=(16, 8))
sns.barplot(x='category2', y='count', data=top_20_count_df, palette='husl')
plt.title("카테고리2의 항목 순위", fontsize=20)
plt.xlabel('카테고리2', fontsize=14)
plt.ylabel('빈도수', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

- 거의 14만 개의 제품이 운동복.
- 운동복, 화장품, 상의와 블라우스, 신발이 가장 많은 수 기록.

In [ ]:
# category3의 빈도수 계산
count = df['category3'].value_counts()

# 데이터프레임으로 변환
count_df = count.reset_index()
count_df.columns = ['category3', 'count']
top_20_count_df = count_df.head(20)

plt.figure(figsize=(16, 8))
sns.barplot(x='category3', y='count', data=top_20_count_df, palette='husl')
plt.title("카테고리3의 항목 순위", fontsize=20)
plt.xlabel('카테고리3', fontsize=14)
plt.ylabel('빈도수', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

- 바지, 타이츠, 레깅스, 얼굴, 그리고 기타 카테고리가 하위 카테고리 레벨3에서 가장 많이 반복되는 세 가지 항목.

- 데이터셋에는 화장품, 드레스 및 여성 관련 액세서리와 같이 여성과 관련된 제품이 가장 많이 포함되어 있음.

### Brand

In [ ]:
brands =df['brand_name'].value_counts()
top_10_brands_excluding_first = brands[1:11]
top_10_brands_excluding_first

In [ ]:

# 상위 1개 브랜드를 제외한 상위 10개 브랜드의 빈도수 막대 그래프
plt.figure(figsize=(15, 5))
sns.barplot(x=top_10_brands_excluding_first.index, y=top_10_brands_excluding_first.values, palette='husl')
plt.title('상위 10개 브랜드와 각 브랜드의 항목 수 (1위 제외)')
plt.xlabel('브랜드명')
plt.ylabel('수량')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
#가장 비싼 브랜드 찾기
brand_median_price = df.groupby('brand_name')['price'].median().reset_index()

top_25_brands = brand_median_price.sort_values(by='price', ascending=False).head(25)

In [ ]:
plt.figure(figsize=(10, 8))
sns.barplot(
    data=top_25_brands,
    x='price',
    y='brand_name',
    palette='cool'
)
plt.xlabel('Median Price')
plt.ylabel('')
plt.title('Top 25 Most Expensive Brands')
plt.xlim(0, top_25_brands['price'].max() * 1.1)
plt.show()

In [ ]:
# 'Unknown' 브랜드와 그 외의 브랜드 구분
known_brand = np.log(df.loc[df['brand_name']!='No Brand', 'price']+1)
unknown_brand = np.log(df.loc[df['brand_name']=='No Brand', 'price']+1)

# 그래프 그리기
plt.figure(figsize=(10, 6))
sns.histplot(known_brand, kde=True, label='Known brands', color = 'blue')
sns.histplot(unknown_brand, kde=True, label='Unknown brand', color = 'red')

plt.title('Known Brands Product Price vs Unknown Brand Product Price')
plt.xlabel('log(price+1)')
plt.ylabel('Count')
plt.grid(True)
plt.legend()
plt.show()

### Item Condition

In [ ]:
# item_condition_id 별 빈도수 계산
count = df['item_condition_id'].value_counts()

# 상위 5개 항목으로 제한
count = count.head(5)

# 상위 5개 항목을 데이터프레임으로 변환
count_df = count.reset_index()
count_df.columns = ['item_condition_id', 'count']

plt.figure(figsize=(10, 3))
sns.barplot(x='item_condition_id', y='count', data=count_df,  palette='husl')
plt.title('상품 상태 ID별 수량')
plt.xlabel('상품 상태 ID')
plt.ylabel('수량')
plt.show()

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(10, 5))
sns.boxplot(x='item_condition_id', y='price', data=df, showfliers=False)
plt.show()

### Shipping

In [ ]:
sns.countplot(x='shipping', data=df)

In [ ]:
sns.barplot(x='shipping', y='price', data=df)
plt.title('Mean price for shipping')
plt.show()

### Text Analysis

In [ ]:

# NLTK 불용어 다운로드
nltk.download('stopwords')
nltk.download('punkt')

# 불용어 목록 가져오기
stop_words = set(stopwords.words('english'))

# 불용어 제거 함수
def remove_stopwords(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# 데이터프레임의 'name'과 'item_description' 열에 불용어 제거 적용
df['name'] = df['name'].apply(remove_stopwords)
df['item_description'] = df['item_description'].apply(remove_stopwords)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer1 = TfidfVectorizer(max_features=1000)
vectorizer1.fit(df['name'].values)

df_name_tfidf = vectorizer1.transform(df['name'].values)
# X_test_name_tfidf = vectorizer1.transform(df['name'].values)

In [ ]:
vectorizer1 = TfidfVectorizer(min_df=10, max_features=5000) #ngram_range=(1,4)
vectorizer1.fit(df['item_description'].values)

df_id_tfidf = vectorizer1.transform(df['item_description'].values)
# X_test_id_tfidf = vectorizer1.transform(df['item_description'].values)

In [ ]:
# 스타일 설정
plt.style.use('seaborn')
sns.set_palette("husl")

# 그래프 생성 함수
def create_word_frequency_graph(tfidf_matrix, feature_names, title, n_top_words=20):
    tfidf_sums = np.array(tfidf_matrix.sum(axis=0)).flatten()
    top_indices = tfidf_sums.argsort()[-n_top_words:][::-1]

    top_words = [feature_names[i] for i in top_indices]
    top_tfidf = tfidf_sums[top_indices]

    fig, ax = plt.subplots(figsize=(14, 8))
    bars = ax.bar(range(n_top_words), top_tfidf, color=sns.color_palette("husl", n_top_words))

    # 막대 위에 값 표시
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}',
                ha='center', va='bottom', rotation=0)

    plt.xticks(range(n_top_words), top_words, rotation=45, ha='right')
    plt.xlabel('Words', fontsize=14)
    plt.ylabel('TF-IDF Score', fontsize=14)
    plt.title(title, fontsize=18, fontweight='bold')

    # 격자 추가
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()

# 'name' 필드에 대한 그래프 생성
create_word_frequency_graph(df_name_tfidf, vectorizer1.get_feature_names_out(),
                            "Top 20 Words in Name Field")

# 'item_description' 필드에 대한 그래프 생성
create_word_frequency_graph(df_id_tfidf, vectorizer1.get_feature_names_out(),
                            "Top 20 Words in Item Description Field")

## Basic Model (우선 돌려보기)

In [ ]:
train_model1 = df.copy()
test_model1  = df_test.copy()

In [ ]:
## train_id, test_id는 제거하기
train_model1, _ = drop_id_feature(train_model1, 'train_id')
test_model1, _  = drop_id_feature(test_model1, 'test_id')

In [ ]:
## 숫자행 제외하고 제거하기. (240722 14:47 기준 이상치처리를 하지 않아 숫자형은 item, shipping 뿐)
train_model1 = train_model1.drop(columns=train_model1.select_dtypes(exclude=[np.number]).columns)
test_model1  = test_model1.drop(columns=test_model1.select_dtypes(exclude=[np.number]).columns)
df_dtypes(train_model1)

In [ ]:
## test set : 20%
X_train, X_test, y_train, y_test = split_data_train_test(train_model1, target_feature)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
## Ridge, LinearRegreesion, 의사결정나무회귀 돌려보기
models_result1 = run_models(baseline_models, X_train, X_test, y_train, y_test)

In [ ]:
models_result1
## 아무런 처리를 하지 않고 출력했기 때문에 성능이 매우 안 좋은 것을 확인(R^2가 0.01 수준이다.)

In [ ]:
best_models1 = get_best_model(models_result1, baseline_models, 'Mean Squared Error')
print('Best Model of Baseline Models is:', best_models1.__class__.__name__)
## 0.012인 DecisionTreeRegressor가 높게 나옴.